In [160]:
import tensorflow as tf
import numpy as np
import pandas as pd
import json
import os
import cv2
import numpy as np
import pandas as pd

In [161]:
import cv2
import os
import numpy as np
from PIL import Image

batch_size = 10


def get_label_color_values(label_folder_path):
    label_images_names = os.listdir(label_folder_path)
    rgb_values = get_pixel_RGB_values(label_folder_path + '/' + label_images_names[0])
    
    return rgb_values



def get_pixel_RGB_values(path):
    im = Image.open(path)
    rgb_values = []
    pix = im.load()
    
    return list(set(list(im.getdata())))




def label_image_to_one_hot(image, label_values):
    semantic_map = []
    for colour in label_values:
        # colour_map = np.full((label.shape[0], label.shape[1], label.shape[2]), colour, dtype=int)
        equality = np.equal(image, colour)
        class_map = np.all(equality, axis = -1)
        semantic_map.append(class_map)
    semantic_map = np.stack(semantic_map, axis=-1)

    return semantic_map




def get_input_batches(images_path, labels_path):
    
    def generate_batches(batch_size, label_values):
        images_names = os.listdir(images_path)
        total_no_of_images = len(images_names)

        for index in range(0, total_no_of_images, batch_size):
            images_arr = []
            labels_arr = []
            for image_name in images_names[index : index + batch_size]:
                image = cv2.resize(cv2.imread(images_path + "/" + image_name), (620, 180))
                label = cv2.resize(cv2.imread(labels_path + "/" + image_name), (620, 180))

                image = image / 255.0
                label_ = label_image_to_one_hot(image, label_values)
    
                images_arr.append(image)
                labels_arr.append(label_)

            yield np.asarray(images_arr), np.asarray(labels_arr)
    
    return generate_batches
    
        



In [162]:
label_values = get_label_color_values('/Users/vijay/Downloads/Datasets/data_semantics/training/semantic_rgb')
get_batches = get_input_batches('/Users/vijay/Downloads/Datasets/data_semantics/training/image_2', 
                                '/Users/vijay/Downloads/Datasets/data_semantics/training/semantic_rgb')


num_of_classes = len(label_values)

In [ ]:
## Util functions

In [163]:

############################################

def get_scale_and_gamma_variables(shape):

    scale = tf.Variable(tf.ones(shape))
    offset = tf.Variable(tf.zeros(shape))
    
    return scale, offset



############################################

def batch_normalize(data, shape):

    scale, offset = get_scale_and_gamma_variables(shape)
    mean, var     = tf.nn.moments(data, axes = [0, 1, 2])
    data_norm     = tf.nn.batch_normalization(data, mean, var, scale, offset, 0.05, name = "BN")
    
    return data_norm



############################################

def get_weights_and_bias(wts_shape, wts_name, bias_name):
#     print('entered')
#     wts_init = tf.truncated_normal(wts_shape)
    wts_init = tf.truncated_normal_initializer()
    bias_init = tf.constant_initializer(0.1)
    
    wts = tf.get_variable(name = wts_name, shape = wts_shape, initializer = wts_init)
    bias = tf.get_variable(name = bias_name, shape = [wts_shape[-1]], initializer = bias_init)
    
    return wts, bias



############################################

def conv_layer(data, weights, bias, name):

    with tf.variable_scope(name, reuse=tf.AUTO_REUSE):
        
        conv_res      = tf.nn.conv2d(data, filter = weights, strides = [1, 1, 1, 1], padding = 'SAME')
        conv_res_bias = tf.nn.bias_add(conv_res, bias)
        conv_bn       = batch_normalize(conv_res_bias, conv_res_bias.get_shape().as_list()[-1])
        conv_bn_relu = tf.nn.relu(conv_bn)
        
        return conv_bn_relu



In [ ]:
### ENCODER BLOCK

In [164]:
############################################

def max_pooling_with_arg_max(input_data,  pool_name, filter_size = 2, stride = 2):
    filter_shape = [1, filter_size, filter_size, 1]
    strides = [1, stride, stride, 1]
    
    with tf.variable_scope(pool_name, reuse=tf.AUTO_REUSE):
        _, max_indices = tf.nn.max_pool_with_argmax(input_data, ksize = filter_shape, strides = strides, padding = 'SAME')
        pool_res       = tf.nn.max_pool(input_data, ksize = filter_shape, strides = strides, padding = 'SAME')
        max_indices    = tf.stop_gradient(max_indices)
    
    return pool_res, max_indices



    
############################################

def encoder_block(data, filter_size, no_of_filters, wts_name, bias_name, block_name, no_of_blocks = 0):
    conv_data      = data
    
    for block_no in range(0, no_of_blocks):
        
        data_shape = conv_data.get_shape().as_list()
        wts_shape  = [filter_size, filter_size, data_shape[-1], no_of_filters]
        wt_name    = wts_name + '_' + str(block_no)
        bs_name    = bias_name + '_' + str(block_no)
        wts, bias  = get_weights_and_bias(wts_shape, wt_name, bs_name)
        conv_data  = conv_layer(conv_data, wts, bias, (block_name + '_' + str(block_no)))
    
    pool_res, max_indices = max_pooling_with_arg_max(conv_data, (block_name + 'pool'))
    return pool_res, max_indices




############################################
    
def encoder(inputs, encoder_name):
    with tf.variable_scope(encoder_name, reuse=tf.AUTO_REUSE):
        pool_1, max_indices_1 = encoder_block(inputs, 3, 64, 'w_1', 'b_1', 'en_blk_1', 2)
        pool_2, max_indices_2 = encoder_block(pool_1, 3, 128, 'w_2', 'b_2', 'en_blk_2', 2)
        pool_3, max_indices_3 = encoder_block(pool_2, 3, 256, 'w_3', 'b_3', 'en_blk_3', 3)
        pool_4, max_indices_4 = encoder_block(pool_3, 3, 512, 'w_4', 'b_4', 'en_blk_4', 3)
        pool_5, max_indices_5 = encoder_block(pool_4, 3, 512, 'w_5', 'b_5', 'en_blk_5', 3)
        
        return pool_5, [max_indices_5, max_indices_4, max_indices_3, max_indices_2, max_indices_1]
    
    


In [ ]:
### DECODER BLOCK

In [189]:



############################################

def form_weights_and_get_weights_and_biases(data_shape, no_of_filters, filter_size, wts_name, bias_name):
    wts_shape = [filter_size, filter_size, data_shape[-1], no_of_filters]
    wts, bias = get_weights_and_bias(wts_shape, wts_name, bias_name)
    return wts, bias



def flat_the_shape(shape_of_conv_out):

    neurons = 1
    for num in shape_of_conv_out:
        if num is not None:
            neurons = neurons * num

    return neurons



############################################

# https://github.com/Pepslee/tensorflow-contrib/blob/master/unpooling.py
def max_unpooling(input_data, unpool_indices, name):
    
    with tf.variable_scope(name, reuse=tf.AUTO_REUSE):
#         print('unpool indices ' + str(unpool_indices.get_shape().as_list()))
#         print('input_data ' + str(input_data.get_shape().as_list()))
        stride = 2
        input_shape = tf.shape(input_data)
        print('input_shape is ' + str(input_shape))
        print('--------------------------------')

        output_shape = [input_shape[0], input_shape[1] * stride, 
                        input_shape[2] * stride, input_shape[3]]
    #     output_shape = [batch_size, input_shape[1] * stride, input_shape[2] * stride, input_shape[3]]
        print('output shpae ' + str(output_shape))

    #     output_shape_flatten = [output_shape[0], flat_the_shape(output_shape[1:])]
        output_shape_flatten = [output_shape[0], output_shape[1] * output_shape[2] * output_shape[3]]
        input_shape_flatten = tf.reduce_prod(input_shape)
#         print('input shape ' + str(input_shape))
#         print('flat inpyt '  + str(input_shape_flatten))
        input_rs = tf.reshape(input_data, [input_shape_flatten])
        batch_range = tf.reshape(tf.range(tf.cast(output_shape[0], tf.int64), dtype=unpool_indices.dtype), shape=[input_shape[0], 1, 1, 1])
#         batch_range = tf.reshape(tf.range(batch_size, dtype=unpool_indices.dtype), shape=[batch_size, 1, 1, 1])
#         print('batch range is ' + str(batch_range.get_shape().as_list()))
        b = tf.ones_like(unpool_indices) * batch_range
#         b = tf.ones_like(unpool_indices)
#         print('b ' + str(b.get_shape().as_list()))
        b1 = tf.reshape(b, [input_shape_flatten, 1])
        ind_ = tf.reshape(unpool_indices, [input_shape_flatten, 1])
        ind_ = tf.concat([b1, ind_], 1)
#         print('shape of ind_ ' + str(ind_.get_shape().as_list()) + ' ' + str(input_rs.get_shape().as_list()))
#         ret = tf.scatter_nd(ind_, input_rs, shape = output_shape_flatten)
        ret = tf.scatter_nd(ind_, input_rs, shape = tf.cast(output_shape_flatten, tf.int64))
        print('ret shape ' + str(ret.get_shape().as_list()))
        print('output shape ' + str(output_shape))
        ret = tf.reshape(ret, output_shape)
        
        set_input_shape = input_data.get_shape().as_list()
        set_output_shape = [set_input_shape[0], set_input_shape[1] * 2, set_input_shape[2] * 2, set_input_shape[3]]
        ret.set_shape(set_output_shape)
#         print('ret ' + str(ret.get_shape().as_list()))
#         ret = tf.reshape(ret, (set_output_shape))
        return ret


############################################

def decoder_block(data, unpool_indices, shrink_no_of_filters, no_of_filters, block_name, wts_name, bias_name, 
                                                                   no_of_blocks = 0):
#     unpool_conv_data = data
    filter_size = 3
    with tf.variable_scope(block_name, reuse=tf.AUTO_REUSE):
        unpool_conv_data = max_unpooling(data, unpool_indices, 'unpool')
        
        for block_no in range(0, no_of_blocks):
            
            data_shape = unpool_conv_data.get_shape().as_list()
#             print('data _sahpe ' + str(data_shape))
            wt_name    = wts_name + '_' + str(block_no)
            bs_name    = bias_name + '_' + str(block_no)
            
            if block_no  == no_of_blocks - 1 and shrink_no_of_filters:
                wts_shape  = [filter_size, filter_size, data_shape[-1], (no_of_filters / 2)]  
#                 print('wts_shape ' + str(wts_shape))
                wts, bias = get_weights_and_bias(wts_shape, wt_name, bs_name)
                unpool_conv_data = conv_layer(unpool_conv_data, wts, bias, ('deconv_' + str(block_no)))
                return unpool_conv_data
            else:
                wts_shape  = [filter_size, filter_size, data_shape[-1], no_of_filters]
#                 print('wts_shape ' + str(wts_shape))
                wts, bias = get_weights_and_bias(wts_shape, wt_name, bs_name)
                unpool_conv_data = conv_layer(unpool_conv_data, wts, bias, ('deconv_' + str(block_no)))
                return unpool_conv_data
            
#             wts, bias = get_weights_and_bias(wts_shape, wt_name, bs_name)
#             unpool_conv_data = conv_layer(unpool_conv_data, wts, bias, ('deconv_' + str(block_no)))
        
#         return unpool_conv_data


    

    
############################################

def deconv_last_step(inputs, no_of_filters, filter_size, block_name, wts_name, bias_name):
    data_shape = inputs.get_shape().as_list()
    with tf.variable_scope(block_name, reuse=tf.AUTO_REUSE):
        wts_shape      = [filter_size, filter_size, data_shape[-1], no_of_filters]
        wts, bias      = get_weights_and_bias(wts_shape, wts_name, bias_name)
        last_conv      = tf.nn.conv2d(inputs, filter = wts, strides = [1, 1, 1, 1], padding = 'SAME')
        last_conv_bias = tf.nn.bias_add(last_conv, bias)
        return tf.identity(last_conv_bias)
    

    
    
############################################

def decoder(inputs, max_indices, decoder_name):
    upsample_res_1 = decoder_block(inputs, max_indices[0], False, 512, 'de_blk_5', 'de_w_5', 'de_b_5', 3)
    upsample_res_2 = decoder_block(upsample_res_1, max_indices[1], True, 512, 'de_blk_4', 'de_w_4', 'de_b_4', 3)
    upsample_res_3 = decoder_block(upsample_res_2, max_indices[2], True, 256, 'de_blk_3', 'de_w_3', 'de_b_3', 3)
    upsample_res_4 = decoder_block(upsample_res_3, max_indices[3], True, 128, 'de_blk_2', 'de_w_2', 'de_b_2', 2)
    upsample_res_5 = decoder_block(upsample_res_4, max_indices[4], True, 64, 'de_blk_1', 'de_w_1', 'de_b_1', 2)
    logits         = deconv_last_step(upsample_res_5, num_of_classes, 1, 'last_conv', 'l_w', 'l_b')
    return logits




In [ ]:
## Training

In [166]:
X       = tf.placeholder(tf.float32, shape = [None, 180, 620, 3], name = 'x_rs')
Y       = tf.placeholder(tf.float32, shape = [None, 180, 620, num_of_classes])
dropout = tf.placeholder(tf.float32, name = 'dropout')


def encode_and_then_decode(inputs):
    pool_5, max_indices_list = encoder(inputs, 'encoder')
#     print('pool _ 5 ' + str(pool_5.get_shape().as_list()))
    logits      = decoder(pool_5, max_indices_list, 'decoder')
    return logits



##########################################################
    
def logistic_loss(logits, labels, n_classes):
    
    with tf.variable_scope('logistic_loss'):
        
        reshaped_logits = tf.reshape(logits, (-1, n_classes))
        reshaped_labels = tf.reshape(labels, (-1, n_classes))
        entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=reshaped_logits,
                                                          labels=reshaped_labels)
        loss = tf.reduce_mean(entropy, name='logistic_loss')
        return loss





def train_optimizer_and_get_accuracy():
    with tf.variable_scope('optimizer', reuse = tf.AUTO_REUSE):
        logits    = encode_and_then_decode(X)
        cost      = logistic_loss(logits, Y, num_of_classes)
    #     cost      = tf.reduce_mean(tf.square(Y - logits))
        optimizer = tf.train.AdamOptimizer(learning_rate = 0.001).minimize(cost)

        return logits, cost, optimizer


def run_SegNet_model():
    with tf.Session() as session:
        logits, loss, optimizer = train_optimizer_and_get_accuracy()
        
        session.run(tf.global_variables_initializer())
        saver = tf.train.Saver()
        rows = np.shape(X)[0]
        for epoch in range(0, 1):
            images_arr, labels_arr = next(get_batches(batch_size, label_values))
            print(np.shape(images_arr), np.shape(labels_arr))
            _, loss_ = session.run([optimizer, loss], feed_dict = {X : images_arr ,
                                                                   Y : labels_arr, 
                                                                   dropout : 0.5})
                
            print(epoch, loss_)
        save_path = saver.save(session, "./SS_SEGNET_model")
        
    return logits


In [143]:
180 * 620 * 3

334800

In [ ]:
Segnet_logits = run_SegNet_model()

In [ ]:
with tf.Session() as sess:
    
    var = tf.Variable(tf.truncated_normal([6, 6, 6, 6], stddev = 0.02))
    sess.run(tf.global_variables_initializer())
    batch_range = tf.reshape(tf.range(6, dtype=tf.int32),
                                 shape=[6, 1, 1, 1])
    print(batch_range.get_shape().as_list())
    one = tf.ones_like(var)
    print(one.get_shape().as_list())
    b = one * batch_range
    print(b.get_shape().as_list())
